In [11]:
# 导入包(PypI)
import os
import re
import time
import json
import random
import pymongo
import pandas as pd

from PIL import Image
from io import BytesIO
from datetime import datetime
from tqdm.notebook import tqdm

from scrapy.selector import Selector
from scrapy.http import TextResponse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import requests

In [12]:
# 进入facebook网页，登录账号
# 配置Chrome浏览器
service = Service('/Users/apple/.wdm/drivers/chromedriver/mac64/128.0.6613.138/chromedriver-mac-x64/chromedriver')
browser = webdriver.Chrome(service=service)


browser.get('https://www.facebook.com/@NYUShanghai')

wait = WebDriverWait(browser, 60)  # 设置最长等待时间为60秒

In [13]:
def scrape_facebook(num_posts_to_scrape):
    # Initialize storage lists
    post_number = []
    content_list = []
    reactions_list = []
    media_type_list = []
    image_url_list = []
    video_url_list = []
    article_url_list = []
    processed_posts = set()  # Track unique posts by unique identifiers

    wait_time = random.uniform(3, 5)  # Random wait time for scrolling
    collected_posts = 0  # Counter for collected posts

    def fetch_posts():
        """Fetch all currently available posts dynamically."""
        response = TextResponse(
            url=browser.current_url,
            body=browser.page_source.encode('utf-8'),
            encoding='utf-8'
        )
        return Selector(response).xpath('//div[@class="x1yztbdb"]/following-sibling::div[1]/div')

    def get_unique_post_id(post):
        """Generate a unique identifier for a post."""
        return post.xpath('.//@data-testid').get() or post.xpath('.//@id').get() or post.xpath('.//text()').get()

    parent_divs = fetch_posts()

    while collected_posts < num_posts_to_scrape:
        parent_divs = fetch_posts()  # Refresh posts dynamically

        for i in range(len(parent_divs)):
            if collected_posts >= num_posts_to_scrape:
                break

            post = parent_divs[i]
            unique_id = get_unique_post_id(post)
            
            if not unique_id or unique_id in processed_posts:
                print(f"Skipping already processed post: {unique_id}")
                continue

            print(f"\n================= Processing Post {collected_posts + 1} ==================")
            processed_posts.add(unique_id)  # Add to processed set
            post_number.append(collected_posts + 1)

            # Scroll into view
            post_element = browser.find_element(
                By.XPATH,
                f"(//div[@class='x1yztbdb']/following-sibling::div[1]/div)[{i + 1}]"
            )
            browser.execute_script("arguments[0].scrollIntoView(true);", post_element)
            time.sleep(random.uniform(3,5))  # Allow content to load

            # Refresh current post reference
            parent_divs = fetch_posts()
            post = parent_divs[i]

            ### --- Extract Post Text ---
            post_text = post.xpath('.//div[@data-ad-comet-preview="message"]')
            text = post_text.xpath('.//text()').getall()
            cleaned_text = ' '.join([t.strip() for t in text if t.strip()])
            content_list.append(cleaned_text if cleaned_text else None)
            print(f"Text: {cleaned_text}")

            ### --- Extract Video URLs ---
            video_src = post.xpath('.//video[@src]').get()
            if video_src:
                video_url_list.append(video_src)
                media_type_list.append("video")
                image_url_list.append(None)
                article_url_list.append(None)
                print(f"Found Video URL: {video_src}")
            else:
                video_url_list.append(None)
                media_type_list.append(None)

                ### --- Extract Image URLs ---
                image_elements = post.xpath('.//img[@src]')
                img_lst_per_post = [
                    img.xpath('./@src').get()
                    for img in image_elements
                    if img.xpath('./@src').get() and "scontent" in img.xpath('./@src').get()
                ]
                img_lst_per_post = list(set(img_lst_per_post))
                
                if img_lst_per_post:
                    image_url_list.append(img_lst_per_post)
                    media_type_list[-1] = "image"
                    article_url_list.append(None)
                    print(f"Found Image URLs: {img_lst_per_post}")
                else:
                    image_url_list.append(None)
                    
                    ### --- Extract Article URLs ---
                    article_element = post.xpath('(.//a[contains(@rel, "nofollow") and contains(@rel, "noreferrer")])[1]')
                    article_href = article_element.xpath('./@href').get()
                    if article_href and 'http' in article_href:
                        article_url_list.append(article_href)
                        media_type_list[-1] = "article"
                        print(f"Article URL: {article_href}")
                    else:
                        article_url_list.append(None)
                        print("No Article Found.")

            ### --- Extract Reactions ---
            reaction_element = post.xpath('.//span[@class="xt0b8zv x1e558r4"]/text()').get()
            reactions_list.append(reaction_element.strip() if reaction_element else None)

            # ### --- Extract Comments ---
            # comment_element = post.xpath('.//span[contains(@class, "xdj266r")]/text()').get()
            # if comment_element:
            #     match = re.search(r'\d+', comment_element)
            #     comments_list.append(match.group() if match else None)
            # else:
            #     comments_list.append(None)

            collected_posts += 1

        # Scroll down to load more posts
        browser.execute_script("window.scrollBy(0, 500);")
        time.sleep(random.uniform(2, 4))

    ### --- Download Images ---
    print("\nStarting Image Download...")
    image_download_bar = tqdm(total=len(image_url_list), desc="Downloading Images")
    for idx_post, image_url_post in enumerate(image_url_list):
        if image_url_post is None:
            image_download_bar.update(1)
            continue
        
        folder_name = f"data/{idx_post}"
        os.makedirs(folder_name, exist_ok=True)
        for idx_image, url in enumerate(list(set(image_url_post))):
            image_filename = f"{folder_name}/image_{idx_image}.png"
            try:
                response = requests.get(url, timeout=10)
                with open(image_filename, 'wb') as f:
                    f.write(response.content)
            except Exception as e:
                print(f"Error Downloading Image: {url}, Error: {e}")
        image_download_bar.update(1)
    image_download_bar.close()

    ### --- Save Data to CSV ---
    df = pd.DataFrame({
        'Row Number': post_number,
        'Content': content_list,
        'Likes': reactions_list,
        'Images': image_url_list,
        'Video': video_url_list,
        'Article': article_url_list,
        'Post Type': media_type_list
    })
    df.to_csv('scraped_facebook_data.csv', encoding='utf-8-sig', index=False)
    print("Data has been saved to 'scraped_facebook_data.csv'")
    return df

In [14]:
scarped_df = scrape_facebook(10)


================= Processing Post 1 ==================
Text: What happens when motion capture, machine learning, and creative minds collide? The "Duets on AI and Dance" workshop at NYU Shanghai explored how human and machine perspectives can interact, coexist, and occasionally conflict within the context of dance. The event was led by Prof. Jung Hyun Moon from Interactive Media Arts (IMA) and Prof. Siye Tao from Performing Arts. It was supported by Prof. Gottfried Haider and the ml5 team, and partially funded by a grant from the Shang … See more
Found Image URLs: ['https://scontent-lga3-2.xx.fbcdn.net/v/t39.30808-6/472248421_1078144280958057_429744550517440625_n.jpg?_nc_cat=109&ccb=1-7&_nc_sid=127cfc&_nc_ohc=xVlymtF-I_cQ7kNvgFKjpf2&_nc_zt=23&_nc_ht=scontent-lga3-2.xx&_nc_gid=A3DF4s5cqY5D8JYmZ072saN&oh=00_AYCAJY5BAgCqPxEC0KC8E78CsAIMRjd6EWZAB2iVkBedEg&oe=67810B8D', 'https://scontent-lga3-2.xx.fbcdn.net/v/t39.30808-6/472068639_1078144237624728_3157574844650450263_n.jpg?stp=dst-jpg_p960x

Data has been saved to 'scraped_facebook_data.csv'
